In [ ]:
from BetaPose import data_io

In [ ]:
from BetaPose import data_io

In [ ]:
# import numpy as np 
# import h5py as h5 
# import os


matrix = np.arange(100*32*32*32).reshape((100,32,32,32))

targetval = np.arange(100)

# # print(matrix)
with h5.File("/tmp/mytestfile.h5", "w") as file1:
  print(matrix.shape)
  dset = file1.create_dataset("train_x", matrix.shape, dtype='i4', data=matrix, compression="gzip", maxshape=(None, 32,32,32))
  targetvals = file1.create_dataset("train_y", targetval.shape, dtype='i4', data=targetval, compression="gzip", maxshape=(None))


# HDF5 is 187 times smaller than pickle 
dt = h5.special_dtype(vlen=np.dtype(float))
data = np.array([(1, 1.1, np.arange(100).reshape((10,10))), (2, 2.2, np.arange(100,200).reshape((10,10)))], 
                dtype=[('col1', 'i4'), ('col2', 'f4'), ('col3', dt)])


class hdf_operator:
  def __init__(self, filename, new=False): 
    if new or (not os.path.isfile(filename)): 
      self.hdffile = h5.File(filename, "w")
    else: 
      self.hdffile = h5.File(filename, "a")
    
  def list_datasets(self):
    """
    List all available datasets in the HDF5 file.
    """
    datasets = []
    def find_datasets(name, obj):
      if isinstance(obj, h5.Dataset):
        datasets.append(name)
    self.hdffile.visititems(find_datasets)
    return datasets
  
  def data(self, key):
    dset = self.hdffile[key]
    return dset
  def close(self):
    self.hdffile.close()
  
  def column_names(self, dataset_name):
    """
    Get the column names of a dataset.
    Args:
    dataset_name (str): The name of the dataset.
    Returns:
    list of str: A list of column names for the specified dataset.
    """
    dataset = self.data(dataset_name)
    if isinstance(dataset.dtype, np.dtype):
      return dataset.dtype.names
    else:
      raise ValueError(f"{dataset_name} is not a structured dataset with column names.")
  
  def mask_entries(self, dataset_name, boolean_mask):
    """
    Delete a set of entries in a given dataset using a boolean array.
    NOTE: might be slow when dealing with ultra large files
    Args:
    dataset_name (str): The name of the dataset.
    boolean_mask (array-like): A boolean array indicating which entries to keep.
    """
    boolean_mask = np.bool_(boolean_mask)
    if dataset_name not in self.list_datasets(): 
      raise Exception(f"Not found any dataset named {dataset_name}")
      
    # Retrieve the dataset
    dataset = self.data(dataset_name)
    shape_before = dataset.shape; 
    
    # Create a new dataset without the specified entries
    new_data = dataset[:][boolean_mask]
    new_dataset = self.create_dataset(f"{dataset_name}_temp", new_data)
    shape_after = new_dataset.shape; 
    # Copy attributes from the original dataset to the new one
    for key, value in dataset.attrs.items():
      new_dataset.attrs[key] = value

    # Remove the original dataset and rename the new one
    self.hdffile.pop(dataset_name, None);
    self.hdffile.move(f"{dataset_name}_temp", dataset_name); 
    print(f"Successfully masked {np.count_nonzero(boolean_mask)} entries; Shape: {shape_before} -> {shape_after}")
    
  def remove_entry(self, dataset_name, index):
    """
    Remove an entry from the specified dataset by index.

    Args:
    dataset_name (str): The name of the dataset.
    index (int): The index of the entry to remove.
    """
    dataset = self.data(dataset_name)
    shape_before = dataset.shape; 
    data = np.asarray(dataset)
    data = np.delete(data, index, axis=0)
    # Resize the dataset and overwrite with the new data
    dataset.resize(data.shape)
    dataset[...] = data
    shape_after = dataset.shape; 
    print(f"Successfully Delete the entry {index}; Shape: {shape_before} -> {shape_after}")

  def create_dataset(self, data_key, thedata, columns=[], **kwarg): 
    theshape = thedata.shape; 
    maxshape = [i for i in theshape]; 
    maxshape[0] = None; 
    maxshape = tuple(maxshape); 
    dset = self.hdffile.create_dataset(data_key, data=thedata, compression="gzip", maxshape=maxshape, **kwarg)
    print(f"Created Dataset: {data_key}"); 
    return dset
  
  def create_table(self, data_key, thedata, columns=[], **kwarg): 
    if thedata.dtype.names: 
      names = [name.encode('utf-8') for name in thedata.dtype.names]
    else: 
      names = columns; 
    self.create_dataset(data_key, thedata, **kwarg); 
    if len(columns) > 0: 
      dset.attrs['columns'] = names
      
  def delete_dataset(self, dataset_name):
    if dataset_name in self.hdffile:
      self.hdffile.pop(dataset_name, None); 
      print(f"Dataset '{dataset_name}' has been deleted.")
    else:
      print(f"Warning: Dataset '{dataset_name}' does not exist.")

  
  def append_entry(self, dataset_name, newdata):
    dset = self.data(dataset_name);
    current_shape = dset.shape;
    # Calculate the new shape after appending the new data
    new_shape = (current_shape[0] + newdata.shape[0], *current_shape[1:])
    # Resize the dataset to accommodate the new data
    dset.resize(new_shape)
    # Append the new data to the dataset
    dset[-newdata.shape[0]:] = newdata
    print(f"Appended {newdata.shape[0]} entries to {dataset_name}")
    
  def draw_structure(self):
    print("####### HDF File Structure #######")
    def print_structure(name, obj):
      if isinstance(obj, h5.Group):
        print(f"$ /{name:10s}/")
      else:
        print(f"$ /{name:10s}: Shape-{obj.shape}")
    self.hdffile.visititems(print_structure)
    print("##### END HDF File Structure #####")
    
  def alter_entry(self, dataset_name, index, new_data):
    """
    Alter a specific entry in the dataset by index.
    Args:
      dataset_name (str): The name of the dataset.
      index (int): The index of the entry to be modified.
      new_data (tuple): The new data to replace the existing entry.
    """
    dataset = self.data(dataset_name)
    if 0 <= index < len(dataset):
      dataset[index] = new_data
    else:
      print(f"Index {index} is out of range for dataset '{dataset_name}'.")
    
  
print("################")
hdf = io.hdf_operator("/tmp/mytestfile.h5")
hdf.create_table("test", data, columns=[])
hdf.create_table("test2", matrix)

# Entry Addition
hdf.append_entry("train_x", matrix)

# Entry Deletion
hdf.mask_entries("train_x", np.random.random(size=200)>0.1)
hdf.remove_entry("train_x", 10)

# Entry Alternation
hdf.alter_entry("train_x",1, (np.zeros((32,32,32))+1.5).astype(float))


hdf.append_entry("test", data)
hdf.append_entry("test", data)

hdf.column_names("test")
x = hdf.data("test")
hdf.draw_structure()

hdf.delete_dataset("test2")

hdf.draw_structure()

datasets = hdf.list_datasets()
names = hdf.column_names("train_x")
print(names)

hdf.close()

In [15]:
print(f"{15:10d}")

        15


In [58]:
import h5py
import numpy as np

# Create a structured array
data = np.array([(1, 1.1, np.arange(100).reshape((10,10))), (2, 2.2, np.arange(100,200).reshape((10,10)))], 
                dtype=[('col1', 'i4'), ('col2', 'f4'), ('col3', h5py.vlen_dtype(np.dtype('float64')))])

# Create an HDF5 file
with h5py.File('/tmp/example.h5', 'w') as f:
    # Create a dataset with the structured array
    dset = f.create_dataset('table', data=data)

    # Store the column names as an attribute of the dataset
    column_names = [name.encode('utf-8') for name in data.dtype.names]
    print(column_names)
    dset.attrs['column_names'] = column_names

[b'col1', b'col2', b'col3']


In [1]:
with h5.File("/tmp/mytestfile.h5","r") as file1: 
  print(file1.keys())
  print(file1["train_x"])
  x = list(file1["train_x"])
  print(np.array(x))

NameError: name 'h5' is not defined

In [7]:
np.bool_(np.arange(100))

array([False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [1]:
import os
import numpy as np
import h5py as h5

class hdf_operator:
    def __init__(self, filename, new=False):
        if new or (not os.path.isfile(filename)):
            self.hdffile = h5.File(filename, "w")
        else:
            self.hdffile = h5.File(filename, "a")

    def create_dataset(self, dataset_name, data, dtype=None):
        """
        Create a new dataset with the given data.

        Args:
        dataset_name (str): The name of the dataset.
        data (array-like): The data to store in the dataset.
        dtype (optional): The data type of the dataset.
        """
        # Create a compound data type if not provided
        if dtype is None:
            dt = h5.special_dtype(vlen=np.dtype((float, (10, 10))))
            dtype = np.dtype([('col1', 'i4'), ('col2', 'f4'), ('col3', dt)])

        # Create the dataset
        dataset = self.hdffile.create_dataset(dataset_name, data=data, dtype=dtype, maxshape=(None,) + data.shape[1:])

    # ... (previous methods)

# Usage example
hdf_op = hdf_operator("/tmp/mytestfile.h5", new=True)
dt = h5.special_dtype(vlen=np.dtype((float, (10, 10))))
data = np.array([(1, 1.1, np.arange(100).reshape((10, 10))), (2, 2.2, np.arange(100, 200).reshape((10, 10)))],
                dtype=[('col1', 'i4'), ('col2', 'f4'), ('col3', dt)])
hdf_op.create_dataset("my_dataset", data)
hdf_op.hdffile.close()


TypeError: Unable to convert between src and dst data types (no appropriate function for conversion path)